### 导入运行程序，建立路网、交叉口信号灯，设置到达离去

In [ ]:
from ctmnet2.simulation import Simulator
from ctmnet2.networkcreation import create_from_xml

def neta(Area="",Ver=""):
    U=[1,6]
    D=[3,4]
    sim = create_from_xml("NET-"+Area+Ver+".xml")
    sim.set_arrival(str(U[0]),1600*5)
    sim.set_departure(str(D[0]),1412*3)
    sim.set_arrival(str(U[1]),2000*3)
    sim.set_departure(str(D[1]),1412*5)
    return sim

### 路灯检测数据输入

In [ ]:
import xml.etree.ElementTree as ET
import re

def deta(sim,Area=""):
    sec_tree=ET.parse("sec_det-"+Area+".xml")
    det_tree=ET.parse("detectors-"+Area+".xml")
    secs=sec_tree.getroot()
    dets=det_tree.getroot()
    for sec in secs.iter('section'):
        sec_id=sec.attrib['id']
        det_id=re.findall(r"\d+",sec.find('detectors').text)
        #print(sec_id,det_id)
        n=0
        for i in det_id:
            for det in dets.iter('detector'):
                if det.attrib['id']==i:
                    vol=float(det.find('volume').text)
                    sim.sections[sec_id].cells[int(n)].volume=vol
                    n=n+1
    return sim

### 开始仿真

In [ ]:
def simula(sim,T=600,R='volume',S='cell'):
    result=[]
    r=[]
    s=0
    for i in range(T):
        r=[]
        s=0
        sim.run_single_step()
        if R=='volume':
            V=sim.get_volume(S)
        elif R=='density':
            V=sim.get_density(S)
        elif R=='velocity':
            V=sim.get_velocity(S)
        elif R=='occupancy':
            V=sim.get_occupancy(S)
        if S=='cell':
            for j in range(1,7):
                t=[]
                for k in range(len(V[str(j)])):
                    t.append(V[str(j)][k])
                r=r+[round(i,1) for i in t]
            result.append(r)
        elif S=='section':
            for j in range(1,7):
                s=s+V[str(j)]
            result.append(s/6)
    return result

### 矩阵转置

In [ ]:
def maTrix(matrix):
    T=[]
    for i in range(len(matrix[0])):
        t=[]
        for j in range(len(matrix)):
            t.append(matrix[j][i])
        T.append(t)
    return T

### 提取路网中各路段元胞坐标

In [ ]:
import xml.etree.ElementTree as ET
import re

def coora(Area=""):
    tree = ET.parse("NET-"+Area+".xml")
    root = tree.getroot()
    para = 'coordinates'
    coordinates=[]
    for sec in root.iter('section'):
        sec_id = sec.attrib['id']
        coors = re.findall(r"(\d+\.?\d*, \d+\.?\d*)",sec.find(para).text)
        COORS = []
        COOR = []
        for coor in coors:
            COOR = coor.split(',')
            COOR[0] = float(COOR[0])
            COOR[1] = float(COOR[1])
            COORS.append(COOR)
        coordinates.extend(COORS)
    return coordinates

### 仿真后生成HTML文件

In [6]:
import webbrowser
from IPython.display import IFrame

def visula(Matrix,Coor,Area="",R=""):
    with open("HMWT-"+R+"-"+Area+".html",'r',encoding='utf-8') as file:
        with open("HMWT-"+R+"-"+Area+"+.html",'w',encoding='utf-8') as out:
            load=file.read()
            p1,pr=load.split("元胞坐标")
            out.write(p1+'\n')
            for coor in Coor:
                out.write('{"lat":'+str(coor[0])+',"lng":'+str(coor[1])+'},\n')
            p2,pr=pr.split("元胞流量数据")
            out.write(p2+'\n')
            for t in Matrix:
                out.write(str(t)+',\n')
            out.write(pr)

    return IFrame(src='HMWT-'+R+"-"+Area+'+.html', width=600, height=600)

In [7]:
Area="SQM"
R="velocity"
sim=neta(Area)
sim=deta(sim,Area)
result=simula(sim,300,R)
Result=maTrix(result)
Coor=coora(Area)
visula(Result,Coor,Area,R)